In [167]:
import numpy as np
import importlib
import matplotlib.pyplot as plt
import sklearn.decomposition as decomposition
import sklearn.ensemble as ensemble
import sklearn.svm as svm

In [168]:
import dataGenerator
importlib.reload(dataGenerator)

<module 'dataGenerator' from 'C:\\Users\\18pep\\Documents\\sem_I\\DVaE\\clustering_dimensions_reduction\\dataGenerator.py'>

In [169]:
X,y= (dataGenerator.dataGenerator(500)
     .generate_relevant(0,2.5,1,1.5,5)
     .generate_irrelevant(2,5)
     .generate_correlated(5)
     .generate_redundant(5)
     .build())

In [177]:
X.var(axis=0)>2

array([ True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True])

array([[-2.79320187e+01, -6.07617389e+00, -5.31864327e+00,
         6.26491239e-01, -9.24235186e-01],
       [-2.78909200e+01, -5.48867073e+00, -4.03364110e+00,
        -1.34036189e-01, -2.50840023e-01],
       [-2.85884007e+01, -5.42245716e+00, -3.13617581e+00,
         8.10309051e-01,  1.34366830e+00],
       ...,
       [-2.60753536e+01, -2.37145961e-01,  5.79463228e+00,
        -1.72921412e+00,  5.79166255e+00],
       [ 1.49671951e+02,  4.37165761e+01, -2.59603220e+00,
        -3.78113039e+00,  1.00472226e+00],
       [ 2.09332693e+02,  6.00620968e+01, -8.43476992e+00,
        -1.89871742e-01,  5.77280077e-01]], shape=(500, 5))

In [178]:
import featureSelectionMethods

In [179]:
featureSelectionMethods.featureSelector(n=5).filter(X,y)

[3.87039935e-064 3.56166539e-076 1.57686948e-074 1.62735458e-076
 1.26821043e-067 9.24787729e-001 9.62732056e-001 4.85588601e-001
 2.59316116e-001 4.43108798e-001 3.03381921e-116 1.32259416e-114
 3.56166539e-076 2.97733488e-114 1.10582622e-137 1.25990064e-017
 1.26821043e-067 1.57686948e-074 2.31054584e-046 2.62508093e-026]
